# 생명선 classification

생명선

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tqdm.auto import tqdm

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data

/content/drive/MyDrive/Colab Notebooks/AI_Project/인지프 팀플/classification/data


In [ ]:
df = pd.read_excel('label4.xlsx')
df = df[~df['label'].str.contains("-")]  # Filter out invalid entries

In [ ]:
print(df)

     File Number  life head  heart label
0              1     0    0      0   000
1              2     0    0      0   000
2              5     0    1      0   010
3             12     0    0      1   001
4             13     0    1      0   010
..           ...   ...  ...    ...   ...
974         5353     0    0      1   001
975         5354     1    0      1   101
976         5366     0    0      0   000
977         5367     0    0      0   000
979         5385     0    0      1   001

[925 rows x 5 columns]


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def load_and_preprocess_images(image_df):
    images = []
    for idx, row in tqdm(image_df.iterrows(), total=image_df.shape[0]):
        image_path = os.path.join('./life', f"image{row['File Number']}.png")
        image = Image.open(image_path).resize((128, 128))
        image = np.array(image) / 255.0
        if image.ndim == 2 or image.shape[2] == 1:
            image = np.stack((image,)*3, axis=-1)  # Convert grayscale to RGB
        images.append(image)
    return np.array(images)

In [ ]:
train_images = load_and_preprocess_images(train_df)
test_images = load_and_preprocess_images(test_df)

  0%|          | 0/740 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
train_labels = train_df['life'].values
test_labels = test_df['life'].values

In [ ]:
# Initialize MobileNetV2 model with pretrained ImageNet weights
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 unit for binary classification
])

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(patience=5, verbose=1),
    ModelCheckpoint('life_best_model.h5', monitor="val_binary_accuracy", save_best_only=True, save_weights_only=True)
]

history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    batch_size=32,
    epochs=50,  # May increase if model is not overfitting
    callbacks=callbacks
)

# Load the best weights and unfreeze the base_model for fine-tuning
model.load_weights('life_best_model.h5')
base_model.trainable = True

Epoch 1/50
24/24 [==============================] - 17s 572ms/step - loss: 0.7073 - binary_accuracy: 0.6000 - val_loss: 0.6647 - val_binary_accuracy: 0.6108
Epoch 2/50
24/24 [==============================] - 9s 361ms/step - loss: 0.6633 - binary_accuracy: 0.6351 - val_loss: 0.6455 - val_binary_accuracy: 0.6108
Epoch 3/50
24/24 [==============================] - 12s 489ms/step - loss: 0.6629 - binary_accuracy: 0.6378 - val_loss: 0.6564 - val_binary_accuracy: 0.6108
Epoch 4/50
24/24 [==============================] - 10s 439ms/step - loss: 0.6520 - binary_accuracy: 0.6311 - val_loss: 0.5813 - val_binary_accuracy: 0.6108
Epoch 5/50
24/24 [==============================] - 11s 443ms/step - loss: 0.6416 - binary_accuracy: 0.6351 - val_loss: 0.6359 - val_binary_accuracy: 0.6108
Epoch 6/50
24/24 [==============================] - 12s 502ms/step - loss: 0.6093 - binary_accuracy: 0.6297 - val_loss: 0.5601 - val_binary_accuracy: 0.6108
Epoch 7/50
24/24 [==============================] - 9s 397m

In [ ]:
model.load_weights('life_best_model.h5')

In [ ]:
# Evaluate the model on the test set.
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

6/6 [==============================] - 5s 373ms/step - loss: 0.4286 - binary_accuracy: 0.8703
Test accuracy: 87.03%


In [ ]:
from sklearn.metrics import f1_score

# Predict on the test set
test_predictions = model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions

# Calculate the F1 score
f1_macro = f1_score(test_labels, test_predictions, average='macro')
f1_micro = f1_score(test_labels, test_predictions, average='micro')
f1_weighted = f1_score(test_labels, test_predictions, average='weighted')
f1_none = f1_score(test_labels, test_predictions, average=None)
print(f"F1 Score(macro): {f1_macro}")
print(f"F1 Score(micro): {f1_micro}")
print(f"F1 Score(weighted): {f1_weighted}")
print(f"F1 Score(none): {f1_none}")

6/6 [==============================] - 3s 280ms/step
F1 Score(macro): 0.8604826546003017
F1 Score(micro): 0.8702702702702703
F1 Score(weighted): 0.868672292201704
F1 Score(none): [0.8974359  0.82352941]
